In [1]:
import numpy as np
import pandas as pd
import torch
import textflint
from predict_utils import *

/home/ubuntu/mambaforge/envs/trainbert/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-23 21:02:48.232430: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-23 21:02:50.111305: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-23 21:02:50.112194: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node

In [2]:
test1 = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test_private_expanded.csv')
test2 = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test_public_expanded.csv')
test = pd.concat([test1, test2])
with multiprocessing.Pool(processes=8) as pool:
     text_list = pool.map(normalize, test.comment_text.tolist())

In [3]:
test_y = (test["toxicity"] >= 0.5).astype(int).astype(str).values

In [4]:
json_data = [{"x": a, "y": b} for a, b in zip(text_list, test_y)]

In [4]:
import json
# with open("models/test_list.json", "w+") as f:
#     json.dump(json_data, f)
with open("models/test_list.json", "r") as f:
    json_data = json.load(f)

In [6]:
json_data[0]

{'x': '[ Integrity means that you pay your debts.] Does this apply to President Trump too?',
 'y': '0'}

In [2]:
from textflint.input.component.sample.sa_sample import SASample
from textflint.input.dataset import Dataset
from textflint.generation.transformation.UT.keyboard import Keyboard
from textflint.generation.transformation.UT.spelling_error import SpellingError
from textflint.generation.transformation.UT.swap_syn_wordnet import SwapSynWordNet

In [7]:
sa_ds = Dataset("SA")
sa_ds.load(json_data)
# sa_ds.load_json("models/test_list.json")
tf1_ds = Dataset("SA")
tf2_ds = Dataset("SA")
tf3_ds = Dataset("SA")

tf1 = Keyboard()
tf2 = SpellingError()
tf3 = SwapSynWordNet()

TextFlint: ******Start load!******
  0%|          | 0/194640 [00:00<?, ?it/s]TextFlint: Downloading http://textflint.oss-cn-beijing.aliyuncs.com/download/SPACY_MODEL/model.zip.
100%|██████████| 764M/764M [07:10<00:00, 1.77MB/s]
TextFlint: Unzipping file /home/ubuntu/.cache/textflint/tmpwqo9ulfp to /home/ubuntu/.cache/textflint/SPACY_MODEL.
TextFlint: Successfully saved SPACY_MODEL/model.zip to cache.
/home/ubuntu/mambaforge/envs/trainbert/lib/python3.7/site-packages/spacy/util.py:887: UserWarning: [W095] Model 'en_core_web_lg' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.5.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
100%|██████████| 194640/194640 [08:57<00:00, 362.18it/s] 
TextFlint: 194640 in total, 194640 were loaded successful.
TextFlint: ****

IndexError: list index out of range

In [11]:
from tqdm import tqdm

def transform_dataset(tf, ds, tf_ds):
    for s in tqdm(ds):
        transformed = tf.transform(s)
        if len(transformed) > 0:
            tf_ds.append(transformed[0])

transform_dataset(tf1, sa_ds, tf1_ds)
transform_dataset(tf2, sa_ds, tf2_ds)
transform_dataset(tf3, sa_ds, tf3_ds)

100%|██████████| 194640/194640 [29:12<00:00, 111.08it/s] 


In [14]:
tf1_ds.save_csv("models/test_tf1.csv")

TextFlint: Save samples to models/test_tf1.csv!


In [15]:
tf2_ds.save_csv("models/test_tf2.csv")
tf3_ds.save_csv("models/test_tf3.csv")

TextFlint: Save samples to models/test_tf2.csv!
TextFlint: Save samples to models/test_tf3.csv!
